# Performance Comparison
Compare performance when reading json vs jsonstat formats

In [1]:
# imports
import pandas as pd # Dataframe
import json # Parsing json to object
import requests # Making HTTP get requests
import pycountry
from pyjstat import pyjstat

In [2]:
def download_indicator_json(indicator_name):
  index = 0
  page = 1
  df = pd.DataFrame(columns=["country_region", "iso_code", "year", "value", "unit"])
  while True:
    url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_name}?format=json&per_page=1000&gapfill=N&page={page}" # REPLACE ALL WITH VARIABLE FOR ISO3_CODES
    response = requests.get(url)
    if response.status_code == 200:
      data = json.loads(response.text)
      total_pages = data[0]["pages"]
      print(f"Processing {indicator_name}: Page {page} / {total_pages}")
      for item in data[1]:
        df.loc[index] = { "country_region":item["country"]["value"], "iso_code": item["countryiso3code"], "year": item["date"], "value": item["value"], "unit": item["unit"] }
        index = index + 1
      if (page > total_pages):
        break
      else:
        page = page + 1
    else:
      raise Exception(f"Failed to download indicator {indicator_name}")
  return df

In [3]:
def download_indicator_jsonstat(indicator_name):
    url = f"http://api.worldbank.org/v2/country/all/indicator/{indicator_name}?format=jsonstat&gapfill=N"
    dataset = pyjstat.Dataset.read(url)
    df = dataset.write('dataframe')
    return df

In [4]:
%%timeit -n 1 -r 1
result1 = download_indicator_json("SG.GEN.PARL.ZS")

Processing SG.GEN.PARL.ZS: Page 1 / 17
Processing SG.GEN.PARL.ZS: Page 2 / 17
Processing SG.GEN.PARL.ZS: Page 3 / 17
Processing SG.GEN.PARL.ZS: Page 4 / 17
Processing SG.GEN.PARL.ZS: Page 5 / 17
Processing SG.GEN.PARL.ZS: Page 6 / 17
Processing SG.GEN.PARL.ZS: Page 7 / 17
Processing SG.GEN.PARL.ZS: Page 8 / 17
Processing SG.GEN.PARL.ZS: Page 9 / 17
Processing SG.GEN.PARL.ZS: Page 10 / 17
Processing SG.GEN.PARL.ZS: Page 11 / 17
Processing SG.GEN.PARL.ZS: Page 12 / 17
Processing SG.GEN.PARL.ZS: Page 13 / 17
Processing SG.GEN.PARL.ZS: Page 14 / 17
Processing SG.GEN.PARL.ZS: Page 15 / 17
Processing SG.GEN.PARL.ZS: Page 16 / 17
Processing SG.GEN.PARL.ZS: Page 17 / 17
Processing SG.GEN.PARL.ZS: Page 18 / 17
1min 13s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%%timeit -n 1 -r 1
result2 = download_indicator_jsonstat("SG.GEN.PARL.ZS")

765 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
